CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [ ]:
# !pip install alibi-detect --user

In [1]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

import pytorch_tabular
import sklearn.metrics 
from sklearn.metrics import r2_score

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [2]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
model = pytorch_tabular.tabular_model.TabularModel.load_model('saved_models/part_b1')

dataset2022 =df[df['year']==2022]
pred = model.predict(dataset2022)

target = pred['resale_price']
prediction = pred['resale_price_prediction']

print("R2 score: ", r2_score(target, prediction))


2023-10-13 10:34:12,255 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-10-13 10:34:12,261 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

R2 score:  0.43884704181243805


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [3]:
# TODO: Enter your code here

dataset2023 =df[df['year']==2023]
pred = model.predict(dataset2023)

target = pred['resale_price']
prediction = pred['resale_price_prediction']

print("R2 score: ", r2_score(target, prediction))

Output()

R2 score:  0.16212561515994695


> Did model degradation occur for the deep learning model?


Yes it did. The R^2 value decreased from aproximately 0.439 to 0.162.



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [4]:
# TODO: Enter your code here
categorical_cols = ['month', 'town', 'flat_model_type', 'storey_range']
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']

X = df[categorical_cols + continuous_cols]
y = df['resale_price']
feature_names = X.columns.values
category_map = {}
for i in range(len(X.columns)):
    if X.columns[i] in categorical_cols:
        category_map[i] = df[X.columns[i]].unique().tolist()

train_data = df[df['year'] <= 2019]

dp = 1000

train_data = train_data[:dp]
data_2023 = dataset2023[:dp]

X_train = train_data[continuous_cols + categorical_cols].values
y_train= train_data['resale_price'].values

X_test = dataset2023[continuous_cols + categorical_cols].values
y_test = dataset2023['resale_price'].values

categories_per_feature = {key: None for key in list(category_map.keys())}
cd = TabularDrift(X_train, p_val=.05, categories_per_feature=categories_per_feature)

pred = cd.predict(X_test)
drifted_columns = []
fpred = cd.predict(X_test, drift_type='feature')
for f in range(cd.n_features):
    fn = feature_names[f]
    drift = fpred['data']['is_drift'][f]

    if drift:
        drifted_columns.append(fn)

> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


Concept drift.

> From your analysis via TabularDrift, which features contribute to this shift?


In [5]:
print("Drifted Columns: ", drifted_columns)

Drifted Columns:  ['month', 'town', 'flat_model_type', 'storey_range', 'dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality']


These are the features that contribute to the drift, `month`, `town`, `flat_model_type`, `storey_range`, `dist_to_nearest_stn`, `dist_to_dhoby`, `degree_centrality`, `eigenvector_centrality`

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


Retrain the model with newer data

In [6]:
# TODO: Enter your code here
model.fit(train=df[df['year'] <= 2022])

Global seed set to 42
2023-10-13 10:34:20,587 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 10:34:20,596 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 10:34:20,767 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-13 10:34:20,798 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-10-13 10:34:20,835 - {py

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at c:\Users\XIANG\OneDrive - Nanyang Technological University\Desktop\NTU\Y4S1\CZ4042 Neural Networks\Assignment 1\.lr_find_dc74499b-0745-4d34-9819-a1ad38610b37.ckpt
Restored all states from the checkpoint file at c:\Users\XIANG\OneDrive - Nanyang Technological University\Desktop\NTU\Y4S1\CZ4042 Neural Ne

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-13 10:37:17,357 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 10:37:17,359 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\utilities\cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [9]:
test = df[df['year'] == 2023]

res = model.evaluate(test)
pred = model.predict(test)

target = pred['resale_price']
prediction = pred['resale_price_prediction']
print(f"R^2: {r2_score(target, prediction)}")

Output()

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\core\module.py:493: UserWarning: 
You called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\core\module.py:493: UserWarning: 
You called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable 
one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

C:\Users\XIANG\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       13991292928.0       │
│  test_mean_squared_error  │       13991292928.0       │
└───────────────────────────┴───────────────────────────┘

Output()

R^2: 0.525413150660875


Retraining the model with newer data has led to the R^2 increasing from approximately 0.162 to 0.525

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |